# THE SOLOW MODEL WITH LAND

Imports and set magics:

In [2]:
import numpy as np
from scipy import optimize
import sympy as sm
import matplotlib.pyplot as plt

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# local modules
from modelproject import solow_equation, multi_start

# Model description

We have the following equations in the Solow Model with land:
\begin{aligned}
    Y_{t} &= K^{\alpha}_{t}(A_{t}L_{t})^{\beta}X^{\kappa}, \quad \alpha>0, \beta>0, \kappa>0, \alpha+\beta+\kappa=1 \hspace{4em} &(1)\\
    K_{t+1} &= sY_{t} + (1-\delta)K_{t}, \quad 0 < s < 1, 0 < \delta < 1, \quad K_{0} \text{ given}\hspace{4em} &(2)\\
    L_{t+1} &= (1+n)L_{t}, \quad n≥0,  \quad L_{0} \text{ given}\hspace{4em} &(3)\\
    A_{t+1} &= (1+g)A_{t}, \quad g≥0, \quad A_{0}  \text{ given} \hspace{4em} &(4)
\end{aligned}

1. The first equation is a Cobb-Douglas aggregate production function with land and exogenous constant technical coefficients $\alpha$, $\beta$ and $\kappa$. The input X of land does not carry a time subscript since alnd is in fixed supply. <br>
2. The second equation is the capital accumulation equation, assuming given rates of gross investments in physical capital (now disregarding human kapital) and of depreciation. <br>
3. Equation (3) and (4) assume given growth rates for the labour force, $L_{t}$, and for the labour-augmenting productivity factor, $A_{t}$. Thus n is the growth rate of the labour force, and g reflects the rate of labour-augmenting technological progress. We assume both of these rates are at least zero. <br>

We asumme constant return to scale for $K_{t}$, $L_{t}$ and X. When X is constant over time there is diminishing returns to scale for $K_{t}$ and $L_{t}$.

The competetive factor markets imply that, in each period, each factor will earn its marginal product which depends on the predetermined levels of $K_{t}$, $L_{t}$, $A_{t}$ and X:
\begin{aligned}
    (\frac{K_{t}}{A_{t}L_{t}})^{\alpha-1}(\frac{X_{t}}{A_{t}L_{t}})^{\kappa},  \hspace{4em} &(5)\\
    (\frac{K_{t}}{A_{t}L_{t}})^{\alpha}(\frac{X_{t}}{A_{t}L_{t}})^{\kappa}A_{t},  \hspace{4em} &(6)\\
    (\frac{K_{t}}{A_{t}L_{t}})^{\alpha}(\frac{X_{t}}{A_{t}L_{t}})^{\kappa-1},  \hspace{4em} &(7)\\
\end{aligned}


## Income shares of production factors

In [3]:
import sympy as sm

# Create SymPy symbols for the variables and parameters
Y_t, K_t, X, A_t, L_t, alpha, beta, kappa, r_t, w_t, landrent_t = sm.symbols('Y_t, K_t, X, A_t, L_t, alpha, beta, kappa, r_t, w_t, landrent_t')

# Set the production function
production_function = K_t**alpha * (A_t*L_t)**beta * X**kappa

# Find rent and wages
rent = sm.diff(production_function, K_t)
wages = sm.diff(production_function, L_t)
landrent = sm.diff(production_function, X)

# Replace the production function with Y_t
rent = rent.subs(production_function, Y_t)
wages = wages.subs(production_function, Y_t)
landrent = landrent.subs(production_function, Y_t)

# For display purposes, I create equations for rent and wages
display_rent = sm.Eq(r_t, rent)
display_wages = sm.Eq(w_t, wages)
display_landrent = sm.Eq(landrent_t, landrent)

# Display rent and wages in SymPy format
display(display_rent, display_wages, display_landrent)

Eq(r_t, Y_t*alpha/K_t)

Eq(w_t, Y_t*beta/L_t)

Eq(landrent_t, Y_t*kappa/X)

We have now shown that the income shares of capital, labour and land ($r_{t}K_{t}/Y_{t}$, etc.) are $\alpha$, $\beta$ and $\kappa$, respectively, so the model is one of constant income shares.

## Solving the Steady-State

The Solow model with land can be analyzed directly in terms of the capital-output ratio, $z_{t}≡K_{t}/Y_{t}=k_{t}/y_{t}$, for which we want to establish convergence to a constant steady state level. We will start by using the transition equation for $z_{t}$:
\begin{aligned}
    z_{t+1} &= (\frac{1}{(1+n)(1+g)})^{\beta}[s+(1-\delta)z_{t}]^{1-\alpha}z_{t}^{a} \hspace{4em} &(8)
\end{aligned}

In the Steady-State we have that $z_{t_+1}=z_{t}=z^{*}$. We now isolate $z$:

## Numerical solution

In [4]:
# Defining parameters

s_val = 0.2
n_val = 0.005
g_val = 0.02
delta_val = 0.06
alpha_val = 0.2
beta_val = 0.6
kappa_val = 0.2
A_0 = 1
L_0 =1

In [5]:
# Call the Multi-Start function to find the steady state value of z.
steady_state_z, smallest_residual = multi_start(num_guesses=100,
            bounds = [1e-5, 50],
            fun = solow_equation, 
            args = (s_val, n_val, g_val, delta_val, alpha_val, beta_val), 
            method = 'hybr')

print(steady_state_z)

[2.53914905]


# Further analysis

Make detailed vizualizations of how your model changes with parameter values. 

Try to make an extension of the model. 

In [6]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider

def equation(beta, alpha, delta, n, g, s, z):
    z_next = ((1 / ((1 + n) * (1 + g))) ** beta) * (s + (1 - delta) * z) ** (1 - alpha) * z ** alpha
    return z_next

def plot_graph(beta, alpha, delta, n, g, s):
    z = np.linspace(0, 4, 100)
    z_next = equation(beta, alpha, delta, n, g, s, z)
    
    plt.figure(figsize=(8, 6))
    
    plt.plot(z, z_next, label='Transition equation')
    plt.plot(z, z, 'r--', label='45-degree line')
    
    plt.xlabel('$z_{t}$')
    plt.ylabel('$z_{t+1}$')
    plt.title('Transition equation vs 45-degree line')
    plt.legend()
    plt.xlim(0, 4)
    plt.ylim(0, 4)
    
    plt.show()

# Define sliders with default values
beta_slider = FloatSlider(min=0, max=2, step=0.1, value=0.6, description='beta')
alpha_slider = FloatSlider(min=0, max=1, step=0.1, value=0.2, description='alpha')
delta_slider = FloatSlider(min=0, max=1, step=0.01, value=0.06, description='delta')
n_slider = FloatSlider(min=0, max=1, step=0.001, value=0.005, description='n')
g_slider = FloatSlider(min=0, max=1, step=0.01, value=0.02, description='g')
s_slider = FloatSlider(min=0, max=1, step=0.01, value=0.2, description='s')

# Create interactive plot
interact(plot_graph, beta=beta_slider, alpha=alpha_slider, delta=delta_slider, 
         n=n_slider, g=g_slider, s=s_slider);


interactive(children=(FloatSlider(value=0.6, description='beta', max=2.0), FloatSlider(value=0.2, description=…

# Conclusion

Add concise conclusion. 